In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

## Getting Boroughs of London

In [2]:
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_London_boroughs")
df = df[0]
df.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,194352,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E,25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,369088,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,236687,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,317264,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,317899,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20


In [3]:
df.drop(df.columns.difference(['Borough','Co-ordinates', 'Headquarters']), 1, inplace=True)
df.columns = ["Borough", "Latitude", "Longitude"]
df.head()

,Borough,Latitude,Longitude
0,Barking and Dagenham [note 1],"Town Hall, 1 Town Square",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E
1,Barnet,"North London Business Park, Oakleigh Road South",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W
2,Bexley,"Civic Offices, 2 Watling Street",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E
3,Brent,"Brent Civic Centre, Engineers Way",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W
4,Bromley,"Civic Centre, Stockwell Close",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E


In [4]:
# Delete all the [note x]-strings after Borough name.
for i, s in enumerate(df["Borough"]):
    if(" [" in s):
        num = s.find(" [")
        s = s[:num]
        df.at[i, "Borough"] = s
df.head()

,Borough,Latitude,Longitude
0,Barking and Dagenham,"Town Hall, 1 Town Square",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E
1,Barnet,"North London Business Park, Oakleigh Road South",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W
2,Bexley,"Civic Offices, 2 Watling Street",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E
3,Brent,"Brent Civic Centre, Engineers Way",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W
4,Bromley,"Civic Centre, Stockwell Close",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E


### Adding latitude and longitude

In [5]:
for i, borough in enumerate(df["Borough"]):
    address = f'{borough}, London, Great Britain'
    
    geolocator = Nominatim(user_agent="l_explorer", timeout=None)
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    
    df.at[i, ["Latitude"]] = latitude
    df.at[i, ["Longitude"]] = longitude

df.to_csv("london_boroughs_latlong.csv", index=False)
df.head()


,Borough,Latitude,Longitude
0,Barking and Dagenham,51.5541,0.150504
1,Barnet,51.6531,-0.200226
2,Bexley,51.4417,0.150488
3,Brent,51.5638,-0.27576
4,Bromley,51.4028,0.0148142


## Getting Frankfurt Stadtteile

In [6]:
# df_f means dataframe frankfurt
df_f = pd.read_html("https://de.wikipedia.org/wiki/Liste_der_Stadtteile_von_Frankfurt_am_Main")
df_f = df_f[0]
df_f.head()

,Nr.,Stadtteil,Fläche[3]in km²,Einwohner,Weiblich,Männlich,Deutsche,Ausländer,Ausländerin Prozent,Einwohnerje km²,Ortsbezirk,Stadtgebietseit,Vorherige Zugehörigkeit
0,1,Altstadt,NaN,NaN,NaN,NaN,NaN,NaN,373,8204,01 Innenstadt I,1866[Anm. 1],Freie Stadt Frankfurt
1,2,Innenstadt,NaN,NaN,NaN,NaN,NaN,NaN,468,4430,01 Innenstadt I,1866[Anm. 2],Freie Stadt Frankfurt
2,3,Bahnhofsviertel,NaN,NaN,NaN,NaN,NaN,NaN,54,6570,01 Innenstadt I,1866[Anm. 3],Freie Stadt Frankfurt
3,4,Westend-Süd,NaN,NaN,NaN,NaN,NaN,NaN,275,7538,02 Innenstadt II,1866[Anm. 3],Freie Stadt Frankfurt
4,5,Westend-Nord,NaN,NaN,NaN,NaN,NaN,NaN,293,6249,02 Innenstadt II,1866[Anm. 3],Freie Stadt Frankfurt


In [7]:
df_f.drop(df_f.columns.difference(['Stadtteil','Weiblich', 'Männlich']), 1, inplace=True)
df_f.columns = ["Borough", "Latitude", "Longitude"]
df_f.head()

,Borough,Latitude,Longitude
0,Altstadt,NaN,NaN
1,Innenstadt,NaN,NaN
2,Bahnhofsviertel,NaN,NaN
3,Westend-Süd,NaN,NaN
4,Westend-Nord,NaN,NaN


### Adding latitude and longitude

In [8]:
for i, borough in enumerate(df_f["Borough"]):
    address = f'{borough}, Frankfurt, Germany'
    
    geolocator = Nominatim(user_agent="f_explorer", timeout=None)
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    #print(latitude)
    df_f.at[i, ["Latitude"]] = latitude
    df_f.at[i, ["Longitude"]] = longitude

df_f.to_csv("frankfurt_boroughs_latlong.csv", index=False)
df_f.head()


,Borough,Latitude,Longitude
0,Altstadt,50.1104,8.6829
1,Innenstadt,50.113,8.67434
2,Bahnhofsviertel,50.1077,8.66868
3,Westend-Süd,50.1152,8.66227
4,Westend-Nord,50.1264,8.66792
